In [6]:
import pandas as pd
import os
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.axes as axs

sns.set()


## Import the data using the relvant specifications

In [15]:
# REad the data
df = pd.read_csv(
    "{}/Data/crimes.csv".format("/".join(os.getcwd().split("/")[:-1])),
    delimiter="\t",
    dtype={
        "CrimeID": str,
        "beat": str,
    #     "iucr": str,
        "district": str,
        "casenumber": str,
    #     "date": str,
    #     "block": str,
        "location": str,
    },
)
None

/tmp/ipykernel_50647/3131757411.py:2: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


### Missing values processing 

In [16]:
df

,CrimeID,casenumber,date,block,iucr,primarytype,description,locationdescription,arrest,beat,district,latitude,longitude,location
0,1,JA118920,01/01/2017 01:00:00 AM,023XX N WAYNE AVE,1156,DECEPTIVE PRACTICE,ATTEMPT - FINANCIAL IDENTITY THEFT,RESIDENCE,false,1811,18,41.924198,-87.662071,"(41.924196311, -87.662069166)"
1,2,JA100052,01/01/2017 01:00:00 AM,090XX S BLACKSTONE AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,true,413,4,41.730709,-87.587997,"(41.730707788, -87.587998843)"
2,3,JA100367,01/01/2017 01:00:00 AM,049XX N WHIPPLE ST,0486,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,false,1713,17,41.970776,-87.704872,"(41.97077465, -87.704873143)"
3,4,JA100092,01/01/2017 01:00:00 AM,023XX S CALIFORNIA AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,STREET,false,1034,10,41.848907,-87.695366,"(41.848905706, -87.695364781)"
4,5,JA494707,01/01/2017 01:00:00 AM,032XX N MILWAUKEE AVE,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,APARTMENT,false,1732,17,41.940193,-87.725647,"(41.940192486, -87.725649965)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
730895,730747,JE493505,12/31/2021 12:38:00 AM,005XX E 32ND ST,031A,ROBBERY,ARMED - HANDGUN,RESIDENCE - YARD (FRONT / BACK),False,211,2,41.836071,-87.612816,"(41.83607107, -87.612813156)"
730896,730748,JE493511,12/31/2021 12:41:00 AM,038XX W 68TH PL,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,833,8,41.768700,-87.718941,"(41.76870088, -87.718942599)"
730897,730749,JF100517,12/31/2021 12:41:00 PM,038XX W 83RD PL,0820,THEFT,$500 AND UNDER,RESIDENCE - PORCH / HALLWAY,False,834,8,41.741333,-87.717537,"(41.741332605, -87.717536356)"
730898,730750,JF100660,12/31/2021 12:49:00 AM,075XX S SEELEY AVE,1154,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT $300 AND UNDER,RESIDENCE,False,611,6,41.756420,-87.674522,"(41.756418241, -87.674522275)"


##### Initial misisng values check

In [18]:
# simple missing values check
df.isna().sum()


CrimeID                    0
casenumber                 3
date                       2
block                      0
iucr                       3
primarytype                3
description                3
locationdescription     3434
arrest                     3
beat                       3
district                   3
latitude               11583
longitude              11581
location               11581
dtype: int64

remove all instances that do not have a casenumber - resulting in 3 instances removed from the database


In [19]:
# those with missing casenumbers are unworkable 
df.drop(df[df.casenumber.isna()].index, axis=0, inplace=True)

In [20]:
df.isna().sum()

CrimeID                    0
casenumber                 0
date                       0
block                      0
iucr                       0
primarytype                0
description                0
locationdescription     3431
arrest                     0
beat                       0
district                   0
latitude               11580
longitude              11580
location               11580
dtype: int64

##### Duplicates

In [28]:
df.drop_duplicates(keep = 'first', inplace = True, ignore_index = True)

dropping duplicates repoves 149 instances

In [34]:
# doublechecking confirms
df.drop_duplicates(["casenumber"]).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 730748 entries, 0 to 730747
Data columns (total 14 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   CrimeID              730748 non-null  object 
 1   casenumber           730748 non-null  object 
 2   date                 730748 non-null  object 
 3   block                730748 non-null  object 
 4   iucr                 730748 non-null  object 
 5   primarytype          730748 non-null  object 
 6   description          730748 non-null  object 
 7   locationdescription  727321 non-null  object 
 8   arrest               730748 non-null  object 
 9   beat                 730748 non-null  object 
 10  district             730748 non-null  object 
 11  latitude             719169 non-null  float64
 12  longitude            719169 non-null  float64
 13  location             719169 non-null  object 
dtypes: float64(2), object(12)
memory usage: 83.6+ MB


##### Beat & District analysis

See whether beat and district have the right amount of entities (beat should have length 4; district length 2)
Adjust for "0" preceeding

string is na: '', ' ',"N/A"

In [26]:
df[df.isnull() == True]

,CrimeID,casenumber,date,block,iucr,primarytype,description,locationdescription,arrest,beat,district,latitude,longitude,location
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
730895,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
730896,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
730897,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
730898,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### Reset the index